In [1]:
import datetime
import numpy as np
from scipy.stats import poisson
from scipy.signal import savgol_filter

In [2]:
from bokeh.plotting import figure
from bokeh.models import Span

from bokeh.io import show
from bokeh.io import output_notebook

In [3]:
import pandas as pd

In [4]:
def get_poisson_for_day(date, cases, loc = 2):
    """
    Calculate when the infections occurred for a single day
    """
    final = []
    for i in range(14):
        p = poisson.pmf(i, 5, loc = loc)
        final.append((date, p * cases))
        date -= datetime.timedelta(days = 1)
    return final

def get_poisson_for_all_days(dates, cases):
    temp_dict = {}
    for i in range(len(cases)):
        p_data =  get_poisson_for_day(dates[i], cases[i])
        data = {}
        for i in p_data:
            data[i[0]] = i[1]
        for key in data.keys():
            if not temp_dict.get(key):
                temp_dict[key] = data[key]
            else:
                temp_dict[key] += data[key]
    final = []
    for key in sorted(temp_dict.keys()):
        final.append((key, temp_dict[key]))
    return final



In [5]:
def get_lin_vals(cases):
    x = range(len(cases))
    X = list(zip(*[x]))
    xm = sm.add_constant(X)
    model = sm.OLS(cases, xm) 
    result = model.fit()
    predictions = result.get_prediction(xm)
    print(result.pvalues[1])

    pass

In [6]:
output_notebook()


Loading BokehJS ...

In [9]:
DF = pd.read_csv('data/states.csv')
DF['date'] = pd.to_datetime(DF['date'])

wash = DF[DF['state'] == 'Washington']
cases = wash['cases'].tolist()
cases_smooth = savgol_filter(cases, 9, 2)
dates = wash['date'].tolist()
poisson_data = get_poisson_for_all_days(dates, cases)[0:-14]
p = figure(x_axis_type = 'datetime')
#p.line(x = dates, y = cases)
p.line(x = [x[0] for x in poisson_data], y = [x[1] for x in poisson_data],
      color = 'red', legend_label = 'poisson')
p.line(x = dates, y = cases_smooth, color = 'green')
mask_date_start = datetime.datetime(2020, 6, 26)
mask_start = Span(location=mask_date_start,
                            dimension='height', line_color='red',
                            line_dash='dashed', line_width=3)
labor_span = Span(location=datetime.datetime(2020, 9, 7),
                            dimension='height', line_color='red',
                            line_dash='dashed', line_width=3)
p.add_layout(mask_start)
p.add_layout(labor_span)


show(p)
